In [1]:
import yaml
import os
import datetime
import matplotlib.pyplot as plt
import arviz as az
import numpy as np
import pymc3 as pm
import pymc3.distributions.transforms as tr
import shutil
import theano
import theano.tensor as tt
import random
import math
import pandas as pd
import sys
from model import create_model
from helper import initialise, load_data, get_situation, get_outcome
import pickle

In /Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases lat

In [2]:
start_year = 2008
end_year = 2020
n_iter = 500
train_flag = 3 # 1 if train on First Innings, 2 if train on Second Innings, 3 if train on all data
target_accept = 0.95
save_directory = "2008-2020-500-t3-a95"

In [7]:
print("Start Year:", start_year)
print("End Year:", end_year)
print("Number of iterations:", n_iter)
print("Train Flag:", train_flag)
print("Save Directory:", save_directory)
print("Target Accept:", target_accept)

first_innings_data, second_innings_data, both_innings_data = load_data(start_year, end_year)

first_innings_data = both_innings_data[both_innings_data["inning"] == 1]
second_innings_data = both_innings_data[both_innings_data["inning"] == 2]

print("First innings data size:", len(first_innings_data))
print("Second innings data size:", len(second_innings_data))

if train_flag == 1:
    batsmen, bowlers, batsman_index, bowler_index, batsman_stats, bowler_stats, X, id1, id2, noballs_and_wides = initialise(first_innings_data)
elif train_flag == 2:
    batsmen, bowlers, batsman_index, bowler_index, batsman_stats, bowler_stats, X, id1, id2, noballs_and_wides = initialise(second_innings_data)
else:
    batsmen, bowlers, batsman_index, bowler_index, batsman_stats, bowler_stats, X, id1, id2, noballs_and_wides = initialise(both_innings_data)
    
noballs_and_wides_count = sum(list(noballs_and_wides.values()))
total_balls = sum([len(X[i]) for i in range(9)]) + noballs_and_wides_count

print("Number of noballs and wides:", noballs_and_wides_count)
print("Number of balls bowled:", total_balls)

for i in range(9):
    print("Balls in situation %d: %d" % (i+1, len(X[i])))

Start Year: 2008
End Year: 2020
Number of iterations: 500
Train Flag: 3
Save Directory: 2008-2020-500-t3-a95
Target Accept: 0.95
First innings data size: 96126
Second innings data size: 89497
Number of noballs and wides: 6374
Number of balls bowled: 185679
Balls in situation 1: 26920
Balls in situation 2: 35407
Balls in situation 3: 7889
Balls in situation 4: 170
Balls in situation 5: 9743
Balls in situation 6: 32040
Balls in situation 7: 7
Balls in situation 8: 484
Balls in situation 9: 66645


In [35]:
def match(s1, s2):
    if len(s1.split()) != len(s2.split()):
        return False
    if s1.split()[1] == s2.split()[1]:
        a = s1.split()[0]
        b = s2.split()[0]
        if a[0] != b[0]:
            return False
        else:
            if len(a) == 2 and len(b) == 2:
                if a[1] != b[1]:
                    return True
                else:
                    print("hmm", s1, s2)
                    return False
            else:
                return True
                
    return False

In [29]:
print(match("RC Archer", "J Archer"))

False


In [18]:
"JC Archer".split()

['JC', 'Archer']

In [11]:
print("JC Archer" in batsmen)
print("J Archer" in batsmen)

True
True


In [36]:
for i in range(len(batsmen)):
    for j in range(i):
        if match(batsmen[i], batsmen[j]) == True:
            print(batsmen[i], batsmen[j])

DJ Bravo DM Bravo
Yashpal Singh Yuvraj Singh
A Singh Anureet Singh
Anirudh Singh Anureet Singh
Anirudh Singh A Singh
R Sharma RG Sharma
Harmeet Singh Harbhajan Singh
Sunny Singh Swapnil Singh
Harpreet Singh Harbhajan Singh
Harpreet Singh Harmeet Singh
BMAJ Mendis BAW Mendis
Shivam Sharma Sandeep Sharma
R Singh RP Singh
C Ingram CA Ingram
N Naik NS Naik
S Sharma Sandeep Sharma
S Sharma Shivam Sharma
S Singh Swapnil Singh
S Singh Sunny Singh
JM Bairstow J Bairstow
MK Lomror M Lomror
TK Curran T Curran
JC Archer J Archer
SM Curran S Curran
NA Saini N Saini
Shubman Gill S Gill
Shivam Mavi S Mavi
Abhishek Sharma Ankit Sharma
PP Shaw P Shaw
SO Hetmyer S Hetmyer
MM Ali M Ali
Mujeeb Ur Rahman M Ur Rahman
Ravi Bishnoi R Bishnoi
Arshdeep Singh Anureet Singh
Arshdeep Singh A Singh
Arshdeep Singh Anirudh Singh


In [5]:
model = create_model(batsmen, bowlers, id1, id2, X)

In [3]:
with open(save_directory + "/trace.pkl", 'rb') as buff:
    trace = pickle.load(buff)

UnpicklingError: pickle data was truncated

In [ ]:
cutpoints = np.mean(trace.get_values("cutpoints", burn=n_iter//2, combine=True), axis=0)
mu_1 = np.mean(trace.get_values("mu_1", burn=n_iter//2, combine=True), axis=0)
mu_2 = np.mean(trace.get_values("mu_2", burn=n_iter//2, combine=True), axis=0)

delta_1 = np.mean(trace.get_values("delta_1", burn=n_iter//2, combine=True), axis=0)
delta_2 = np.mean(trace.get_values("delta_2", burn=n_iter//2, combine=True), axis=0)
delta = np.greater_equal([i for i in range(9)], 3) * delta_1 + np.greater_equal([i for i in range(9)], 6) * delta_2

In [ ]:
mu_1_sorted = sorted([(mu_1[i], batsmen[i]) for i in range(len(mu_1))])
mu_2_sorted = sorted([(mu_2[i], bowlers[i]) for i in range(len(mu_2))])

In [ ]:
mu_1_sorted[-40:]

In [ ]:
mu_2_sorted[-40:]

In [13]:
summary = az.summary(trace, round_to=5)
summary

/Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/arviz/data/io_pymc3.py:89: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
mu_1[0],0.08807,0.12194,-0.13875,0.31294,0.00240,0.00216,2585.10694,1598.10000,2586.94082,1611.51412,1.00370
mu_1[1],0.01740,0.13091,-0.22283,0.26161,0.00241,0.00293,2954.38010,1000.97700,2939.08527,1653.00502,1.00031
mu_1[2],-0.13733,0.13558,-0.37870,0.12502,0.00236,0.00217,3311.63891,1949.43542,3312.07576,1616.07194,1.00098
mu_1[3],0.04591,0.18563,-0.31420,0.39436,0.00297,0.00499,3902.44468,693.01739,3904.43643,1295.24115,1.00571
mu_1[4],0.18815,0.18685,-0.17592,0.54413,0.00362,0.00322,2664.91923,1683.97800,2668.57806,1470.71497,1.00192
...,...,...,...,...,...,...,...,...,...,...,...
"cutpoints[8,1]",-0.02628,0.30880,-0.56895,0.49404,0.00850,0.00628,1319.32606,1211.14550,1322.82970,1451.23816,1.00126
"cutpoints[8,2]",1.77230,0.31017,1.23193,2.31634,0.00848,0.00604,1336.78200,1319.28773,1343.55346,1521.42066,1.00189
"cutpoints[8,3]",2.26070,0.31183,1.69225,2.80355,0.00845,0.00602,1360.35612,1340.99945,1379.47219,1429.31379,1.00219
"cutpoints[8,4]",2.29859,0.31195,1.77878,2.88999,0.00844,0.00601,1367.37753,1347.90031,1385.54925,1474.01891,1.00211


In [19]:
cutpoints

array([[-3.48253776, -0.15429566,  1.18267412,  1.47971006,  1.53532458,
         3.54749059],
       [-3.5769404 , -1.04274823,  1.19990675,  1.76567548,  1.79415112,
         2.76202898],
       [-3.3931059 , -1.08053988,  0.47524039,  0.87364198,  0.91159129,
         2.13320606],
       [-2.71454048, -1.37594232, -0.43229862,  0.41858305,  1.37854054,
         2.73711903],
       [-1.95824944, -0.25829605,  1.94802993,  2.48879704,  2.55608955,
         3.27998193],
       [-3.0162296 , -0.46096002,  1.23455308,  1.72070751,  1.75208341,
         3.08309999],
       [-2.70784443, -1.39626487, -0.42736428,  0.42506297,  1.37221924,
         2.68584674],
       [-2.70814131, -1.40982226, -0.45170875,  0.4148544 ,  1.3737967 ,
         2.72557126],
       [-1.92622251, -0.01546537,  1.78275056,  2.27231519,  2.3099557 ,
         3.40056832]])

In [44]:
print(batsman_index["V Kohli"])
print(batsman_index["DA Warner"])
print(batsman_index["Mandeep Singh"])
print(batsman_index["RR Pant"])
print(batsman_index["AD Russell"])
print(batsman_index["STR Binny"])
print(batsman_index["MS Dhoni"])
print(batsman_index["R Tewatia"])

0
16
50
31
25
116
62
34


In [11]:
batsman_stats_tmp = batsman_stats.sort_values("SR", ascending=False)
batsman_stats_tmp[batsman_stats_tmp["BF"] >= 50]

,M,Outs,Runs,Avg,BF,SR,4s,6s,Name
41,6,2,179,89.500000,96,186.458333,10,13,KA Pollard
67,7,4,174,43.500000,98,177.551020,17,9,MP Stoinis
27,7,7,212,30.285714,121,175.206612,13,16,N Pooran
39,6,4,161,40.250000,94,171.276596,16,7,AB de Villiers
9,7,7,201,28.714286,122,164.754098,10,16,SV Samson
25,7,7,337,48.142857,211,159.715640,34,12,MA Agarwal
49,3,3,121,40.333333,77,157.142857,12,6,RA Tripathi
14,7,6,233,38.833333,149,156.375839,33,4,SA Yadav
35,6,4,101,25.250000,65,155.384615,12,3,RA Jadeja
4,5,4,82,20.500000,53,154.716981,7,2,PK Garg


In [23]:
player = "KL Rahul"
print(summary.loc["mu_1[%d]" % batsman_index[player]])
print(batsman_stats[batsman_stats["Name"] == player])

mean            0.06002
sd              0.11244
hdi_3%         -0.15723
hdi_97%         0.25891
mcse_mean       0.00234
mcse_sd         0.00224
ess_mean     2313.71880
ess_sd       1255.55390
ess_bulk     2302.21948
ess_tail     1542.05674
r_hat           1.00302
Name: mu_1[24], dtype: float64
    M  Outs  Runs        Avg   BF         SR  4s  6s      Name
24  7     7   389  55.571429  287  135.54007  37  10  KL Rahul


In [27]:
summary.loc["mu_1[116]"]

mean            0.16213
sd              0.20549
hdi_3%         -0.22061
hdi_97%         0.56817
mcse_mean       0.00286
mcse_sd         0.00392
ess_mean     5166.26770
ess_sd       1374.90855
ess_bulk     5146.80490
ess_tail     1511.39599
r_hat           0.99960
Name: mu_1[116], dtype: float64

In [45]:
summary.loc["mu_1[34]"]

mean           -0.03214
sd              0.20590
hdi_3%         -0.39722
hdi_97%         0.35987
mcse_mean       0.00297
mcse_sd         0.00549
ess_mean     4819.92191
ess_sd        704.86567
ess_bulk     4803.73781
ess_tail     1498.93669
r_hat           1.00106
Name: mu_1[34], dtype: float64

In [28]:
batsman_stats

,M,Outs,Runs,Avg,BF,SR,4s,6s,Name
62,12,5,410,82.000000,305,134.426230,22,21,MS Dhoni
16,12,10,703,70.300000,480,146.458333,58,21,DA Warner
25,13,9,514,57.111111,249,206.425703,33,51,AD Russell
17,10,8,456,57.000000,282,161.702128,50,18,J Bairstow
45,14,11,598,54.363636,437,136.842105,50,25,KL Rahul
...,...,...,...,...,...,...,...,...,...
100,1,1,0,0.000000,2,0.000000,0,0,P Krishna
96,1,1,0,0.000000,1,0.000000,0,0,S Kaul
81,1,1,0,0.000000,2,0.000000,0,0,HV Patel
88,1,0,0,0.000000,2,0.000000,0,0,MM Sharma


In [15]:
lst = ["KL Rahul", "HH Pandya", "JC Archer", "HV Patel", "V Kohli", "DA Warner", "RR Pant"]

In [16]:
O = {}
for batsman in lst:
    O[batsman] = []
    for l in range(9):
        for i in range(len(id1[l])):
            if id1[l][i] == batsman_index[batsman]:
                O[batsman].append((l,X[l][i],id2[l][i]))

In [17]:
for batsman in lst:
    print(batsman)
    for l in range(9):
        t = []
        cnt = 0
        runs = 0
        for i in range(len(O[batsman])):
            k = O[batsman][i][1]
            if O[batsman][i][0] == l:
                cnt += 1
                if k == 1:
                    t.append(bowlers[O[batsman][i][2]])
                elif k == 3:
                    runs += 1
                elif k == 4:
                    runs += 2
                elif k == 5:
                    runs += 3
                elif k == 6:
                    runs += 4
                elif k == 7:
                    runs += 6
        
        if cnt == 0:
            continue
        if cnt == 0: SR = 0
        else: SR = runs/cnt * 100
        
        if len(t) == 0: avg = 0
        else: avg = runs
        
        print("Bowlers who took wicket in situation %d:" % (l+1), t)
        print("Balls faced in situation             %d:" % (l+1), cnt)
        print("Average in situation                 %d:" % (l+1), avg)    
        print("Strike rate rate in situation        %d:" % (l+1), SR)   
        print()
    print()

KL Rahul
Bowlers who took wicket in situation 1: []
Balls faced in situation             1: 55
Average in situation                 1: 0
Strike rate rate in situation        1: 129.0909090909091

Bowlers who took wicket in situation 2: []
Balls faced in situation             2: 90
Average in situation                 2: 0
Strike rate rate in situation        2: 140.0

Bowlers who took wicket in situation 3: ['AS Rajpoot']
Balls faced in situation             3: 28
Average in situation                 3: 67
Strike rate rate in situation        3: 239.28571428571428

Bowlers who took wicket in situation 6: ['SN Thakur']
Balls faced in situation             6: 69
Average in situation                 6: 77
Strike rate rate in situation        6: 111.59420289855073

Bowlers who took wicket in situation 9: ['M Prasidh Krishna', 'MM Sharma', 'K Rabada', 'Abhishek Sharma', 'RD Chahar']
Balls faced in situation             9: 45
Average in situation                 9: 48
Strike rate rate in sit

In [4]:
lst = ["K Rabada", "RD Chahar", "M Santner", "Rashid Khan"]

In [5]:
print(bowler_stats[bowler_stats["Name"].isin(lst)])

      M     B  Runs  Wkts      Econ        Avg         SR         Name
89   18   415   540    36  7.807229  15.000000  11.527778     K Rabada
198   4    84    79     4  5.642857  19.750000  21.000000    M Santner
10   46  1095  1205    60  6.602740  20.083333  18.250000  Rashid Khan
45   15   324   359    15  6.648148  23.933333  21.600000    RD Chahar


In [6]:
O = {}
for bowler in lst:
    O[bowler] = []
    for l in range(9):
        for i in range(len(id2[l])):
            if id2[l][i] == bowler_index[bowler]:
                O[bowler].append((l,X[l][i],id1[l][i]))

In [7]:
for bowler in lst:
    print(bowler)
    for l in range(9):
        t = []
        cnt = 0
        runs = 0
        for i in range(len(O[bowler])):
            k = O[bowler][i][1]
            if O[bowler][i][0] == l:
                cnt += 1
                if k == 1:
                    t.append(batsmen[O[bowler][i][2]])
                elif k == 3:
                    runs += 1
                elif k == 4:
                    runs += 2
                elif k == 5:
                    runs += 3
                elif k == 6:
                    runs += 4
                elif k == 7:
                    runs += 6
        if cnt == 0: econ = 0
        else: econ = runs/cnt*6
        print("Wickets taken in situation %d:" % (l+1), t)
        print("Balls bowled in situation  %d:" % (l+1), cnt)
        print("Economy rate in situation  %d:" % (l+1), econ)        

K Rabada
Wickets taken in situation 1: ['PA Patel', 'BB McCullum', 'SK Raina', 'MA Agarwal', 'AM Rahane']
Balls bowled in situation  1: 100
Economy rate in situation  1: 7.68
Wickets taken in situation 2: ['RG Sharma', 'CA Lynn', 'AB de Villiers', 'RV Uthappa']
Balls bowled in situation  2: 36
Economy rate in situation  2: 5.333333333333333
Wickets taken in situation 3: []
Balls bowled in situation  3: 20
Economy rate in situation  3: 12.0
Wickets taken in situation 4: []
Balls bowled in situation  4: 0
Economy rate in situation  4: 0
Wickets taken in situation 5: ['KD Karthik', 'AS Yadav']
Balls bowled in situation  5: 13
Economy rate in situation  5: 7.384615384615385
Wickets taken in situation 6: ['AJ Finch', 'H Viljoen', 'V Kohli', 'AD Nath', 'HH Pandya', 'STR Binny', 'R Parag', 'PA Patel']
Balls bowled in situation  6: 82
Economy rate in situation  6: 9.073170731707318
Wickets taken in situation 7: []
Balls bowled in situation  7: 0
Economy rate in situation  7: 0
Wickets taken in

In [65]:
# Extract 2020 IPL data
columns = ['inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'wide_runs', 'noball_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed','season']
deliveries = []
for file in os.listdir("data/ipl"):
    with open("data/ipl/" + file) as f:
        data = yaml.load(f)
        year = str(data['info']['dates'][0])[:4]
        print(year)
        if year == "2020":
            teams = data["info"]["teams"]
            inning = 1
            for i in data["innings"]:
                innings = list(i.values())[0]
#                 pprint(innings)
                batting_team = innings["team"]
                if teams[0] == batting_team:
                    bowling_team = teams[1]
                else:
                    bowling_team = teams[0]
                for delivery in innings["deliveries"]:
                    deliveries.append(list(list(delivery.values())[0].values()))
                    """
                    delivery_data = list(delivery.values())[0]
                    season = year
                    ball = int(str(list(delivery.keys())[0])[-1])
                    over = int(str(list(delivery.keys())[0])[0:-2])+1
                    batsman = delivery_data["batsman"]
                    bowler = delivery_data["bowler"]
                    non_striker = delivery_data["non_striker"]
                    player_dismissed = None
                    if "wicket" in delivery_data:
                        player_dismissed = delivery_data["wicket"]["player_out"]
                    wide_runs = 0
                    noball_runs = 0
                    if "extras" in delivery_data:
                        if "wides" in delivery_data["extras"]: wide_runs = delivery_data["extras"]["wides"]
                        elif "noballs" in delivery_data["extras"]: noball_runs = delivery_data["extras"]["noballs"]
                    total_runs = delivery_data["runs"]["total"]
                    extra_runs = delivery_data["runs"]["extras"]
                    batsman_runs = delivery_data["runs"]["batsman"]
                    deliveries.append([inning, batting_team, bowling_team, over, ball, batsman, non_striker, bowler, wide_runs, 
                                       noball_runs, batsman_runs, extra_runs, total_runs, player_dismissed, season])

#                     print(deliveries[-1])
                    """
                inning += 1
                bowling_team, batting_team = batting_team, bowling_team
            break

/Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


2013
2015
2011
2017
2010
2016
2020


In [8]:
bowler_stats = bowler_stats.sort_values(by = "Econ")

In [9]:
bowler_stats[bowler_stats["M"] >= 5].head(20)

,M,B,Runs,Wkts,Econ,Avg,SR,Name
145,5,72,71,2,5.916667,35.500000,36.000000,CH Gayle
178,7,156,155,13,5.961538,11.923077,12.000000,L Ngidi
135,12,258,273,22,6.348837,12.409091,11.727273,MA Starc
84,12,222,241,5,6.513514,48.200000,44.400000,J Yadav
10,46,1095,1205,60,6.602740,20.083333,18.250000,Rashid Khan
45,15,324,359,15,6.648148,23.933333,21.600000,RD Chahar
176,8,182,203,9,6.692308,22.555556,20.222222,I Sodhi
83,13,271,306,11,6.774908,27.818182,24.636364,Mohammad Nabi
182,14,230,265,13,6.913043,20.384615,17.692308,M Ali
114,55,1133,1310,48,6.937335,27.291667,23.604167,R Ashwin


In [10]:
bowler_stats[bowler_stats["Name"] == "K Rabada"]

,M,B,Runs,Wkts,Econ,Avg,SR,Name
89,18,415,540,36,7.807229,15.0,11.527778,K Rabada


In [11]:
bowler_stats[bowler_stats["Name"] == "RD Chahar"]

,M,B,Runs,Wkts,Econ,Avg,SR,Name
45,15,324,359,15,6.648148,23.933333,21.6,RD Chahar


In [12]:
noballs_and_wides_count = sum(list(noballs_and_wides.values()))
total_balls = sum([len(X[i]) for i in range(9)]) + noballs_and_wides_count

print("Number of noballs and wides:", noballs_and_wides_count)
print("Number of balls bowled:", total_balls)

for i in range(9):
    print("Balls in situation %d: %d" % (i+1, len(X[i])))

Number of noballs and wides: 2367
Number of balls bowled: 70207
Balls in situation 1: 10329
Balls in situation 2: 13643
Balls in situation 3: 2657
Balls in situation 4: 47
Balls in situation 5: 3490
Balls in situation 6: 12246
Balls in situation 7: 7
Balls in situation 8: 212
Balls in situation 9: 25209


In [27]:
M = matches[((matches["team1"] == "Mumbai Indians") | (matches["team1"] == "Chennai Super Kings")) & ((matches["team2"] == "Mumbai Indians") | (matches["team2"] == "Chennai Super Kings")) & (matches["season"] >= start_year)]

In [31]:
for i in range(len(M)):
    print(M.iloc[i])

season                            2015
city                            Mumbai
date                        2015-04-17
team1                   Mumbai Indians
team2              Chennai Super Kings
toss_winner             Mumbai Indians
toss_decision                      bat
result                          normal
dl_applied                           0
winner             Chennai Super Kings
win_by_runs                          0
win_by_wickets                       6
player_of_match                A Nehra
venue                 Wankhede Stadium
umpire1                   AK Chaudhary
umpire2                      M Erasmus
umpire3                            NaN
Name: 530, dtype: object
season                                        2015
city                                       Chennai
date                                    2015-05-08
team1                          Chennai Super Kings
team2                               Mumbai Indians
toss_winner                    Chennai Super Kings
toss_d

In [15]:
model = create_model(batsmen, bowlers, id1, id2, X)
print("Loaded model.")

Loaded model.


In [16]:
cutpoints = np.loadtxt(save_directory + "/cutpoints.txt")
mu_1 = np.loadtxt(save_directory + "/mu_1.txt")
mu_2 = np.loadtxt(save_directory + "/mu_2.txt")
delta = np.loadtxt(save_directory + "/delta.txt")

In [17]:
mu_1_sorted = sorted([(mu_1[i], batsmen[i]) for i in range(len(mu_1))])
mu_2_sorted = sorted([(mu_2[i], bowlers[i]) for i in range(len(mu_2))])

In [18]:
mu_1_sorted[-40:]

[(0.10916041709637109, 'D Wiese'),
 (0.10934770894644526, 'R Ashwin'),
 (0.10993303999887889, 'JA Morkel'),
 (0.11229339161979769, 'SN Khan'),
 (0.1127478909503026, 'SP Goswami'),
 (0.11437096032879743, 'JJ Roy'),
 (0.11893518085580507, 'KA Pollard'),
 (0.11973728399627434, 'AM Rahane'),
 (0.1197641913558365, 'BA Stokes'),
 (0.12409350999910944, 'SV Samson'),
 (0.12458632572821825, 'MK Pandey'),
 (0.12529378627448076, 'A Ashish Reddy'),
 (0.12581705723980155, 'TM Head'),
 (0.1266853554262674, 'Mandeep Singh'),
 (0.12830466293186954, 'V Shankar'),
 (0.12906243607182608, 'BCJ Cutting'),
 (0.13454642220456814, 'R Parag'),
 (0.13746709676886482, 'RA Tripathi'),
 (0.13786592115940546, 'SP Narine'),
 (0.13961509731866018, 'S Curran'),
 (0.14195755943924404, 'K Gowtham'),
 (0.15013814110151574, 'SPD Smith'),
 (0.15212560290217136, 'AR Patel'),
 (0.15550969171926526, 'KS Williamson'),
 (0.16018892865044904, 'P Shaw'),
 (0.17868610861141282, 'Ankit Sharma'),
 (0.19270783896061308, 'M Ali'),
 (0

In [19]:
mu_2_sorted[-40:]

[(0.09852279207148802, 'STR Binny'),
 (0.09852584144284587, 'Washington Sundar'),
 (0.10219773812386408, 'KH Pandya'),
 (0.10376096637906598, 'S Warrier'),
 (0.10675916523356294, 'GB Hogg'),
 (0.11346459875868388, 'Z Khan'),
 (0.11929593409287752, 'R Tewatia'),
 (0.1199977642744734, 'K Rabada'),
 (0.12801482033450362, 'Harbhajan Singh'),
 (0.13195185903259793, 'MJ McClenaghan'),
 (0.13539201945396168, 'MR Marsh'),
 (0.14494299849733996, 'YS Chahal'),
 (0.14826889109842756, 'R Ashwin'),
 (0.14899405845275598, 'I Sodhi'),
 (0.14968349726117364, 'S Badree'),
 (0.15303535249415953, 'M Morkel'),
 (0.15383183548817486, 'CH Morris'),
 (0.15521730354903285, 'JW Hastings'),
 (0.15536246161082354, 'J Yadav'),
 (0.16339426600638446, 'NB Singh'),
 (0.1662217857020509, 'JJ Bumrah'),
 (0.16631971194855077, 'DJ Muthuswami'),
 (0.17145282390194005, 'SP Narine'),
 (0.17268151503019338, 'N Saini'),
 (0.17912240752051106, 'M Ali'),
 (0.1833297669435748, 'Mohammad Nabi'),
 (0.21035981926431177, 'MA Starc'

In [20]:
trace = pm.load_trace(save_directory + "/trace", model=model)

In [21]:
with open(save_directory + "/trace.pkl", 'wb') as buff:
    pickle.dump(trace, buff)

In [43]:
summary = az.summary(trace, var_names = ["mu_2"], round_to=5)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:91: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [55]:
summary.loc["mu_2[91]"]

mean             0.09982
sd               0.12329
hdi_3%          -0.12528
hdi_97%          0.33650
mcse_mean        0.00124
mcse_sd          0.00088
ess_mean      9886.35422
ess_sd        9886.35422
ess_bulk      9907.51580
ess_tail     15075.01696
r_hat            1.00021
Name: mu_2[91], dtype: float64

In [56]:
summary.loc["mu_2[105]"]

mean            0.26790
sd              0.14849
hdi_3%         -0.01257
hdi_97%         0.54901
mcse_mean       0.00155
mcse_sd         0.00109
ess_mean     9210.88599
ess_sd       9210.88599
ess_bulk     9210.72297
ess_tail     9327.90963
r_hat           1.00006
Name: mu_2[105], dtype: float64

In [33]:
trace["mu_2"][:,bowler_index["K Rabada"]]

array([ 0.24960261,  0.17831207,  0.03588263, ...,  0.15778465,
        0.17572289, -0.08601433])

In [26]:
trace["mu_2"]

array([[ 0.16866099,  0.03143118,  0.10437558, ...,  0.04479149,
        -0.03603981, -0.03153455],
       [ 0.23030471, -0.18718909, -0.00214847, ..., -0.16698039,
         0.10608509,  0.12403798],
       [ 0.35639444,  0.12053095,  0.21637097, ...,  0.18236339,
         0.17278158,  0.05191501],
       ...,
       [ 0.21834728,  0.04902354,  0.10735688, ..., -0.1549285 ,
        -0.23469898,  0.06232831],
       [ 0.22904815, -0.01444475,  0.10469489, ..., -0.13613517,
        -0.14823155,  0.1026084 ],
       [ 0.28200865, -0.22180348,  0.20764808, ..., -0.17745539,
         0.33129564, -0.21698686]])

In [ ]:
p = np.zeros(shape = (len(batsmen),len(bowlers),9,7))
for i in range(len(batsmen)):
    for j in range(len(bowlers)):
        for l in range(9):
            for k in range(7):
                if k == 0:
                    p[i,j,l,k] = 1/(1 + np.exp(-(cutpoints[l,k] - mu_1[i] + mu_2[j] - delta[l])))
                elif k == 6:
                    p[i,j,l,k] = 1 - 1/(1 + np.exp(-(cutpoints[l,k-1] - mu_1[i] + mu_2[j] - delta[l])))
                else:
                    p[i,j,l,k] = 1/(1 + np.exp(-(cutpoints[l,k] - mu_1[i] + mu_2[j] - delta[l]))) - 1/(1 + np.exp(-(cutpoints[l,k-1] - mu_1[i] + mu_2[j] - delta[l])))
                    
# TODO: can be vectorised

In [21]:
# tmp_directory = "2018-2019-20k-iterations"
# cutpoints = np.loadtxt(tmp_directory + "/cutpoints.txt")
# mu_1 = np.loadtxt(tmp_directory + "/mu_1.txt")
# mu_2 = np.loadtxt(tmp_directory + "/mu_2.txt")
# delta = np.loadtxt(tmp_directory + "/delta.txt")

In [32]:
# deliveries_data, matches, first_innings_data, second_innings_data, both_innings_data = load_data(2018, 2019)
# batsmen, bowlers, batsman_index, bowler_index, batsman_stats, bowler_stats, X, id1, id2, noballs_and_wides = initialise(first_innings_data)

In [33]:
# mu_1_sorted = sorted([(mu_1[i], batsmen[i]) for i in range(len(mu_1))])
# mu_2_sorted = sorted([(mu_2[i], bowlers[i]) for i in range(len(mu_2))])

In [73]:
# mu_1_sorted

In [74]:
# mu_2_sorted